# This notebook demonstrates the workflow of using Landsat imagery from Microsoft Planetary Computer to map the area of Lake Mead and plot its changes over time.

The workflow includes the following steps:
1. Find all the Landsat scenes between 1984 and 2020 from Planetary Computer that covers Lake Mead which has less than 10% cloud cover
2. Create a RasterCollection from all the scenes
3. Use a water detection algorithm to extract all water pixels from the Landsat images
4. Use a function to find connected water pixels that represent a water body
5. Find the area of the largest water body in the scene (Lake Mead) and plot the area over time

#### Estimated running time: 3 hours

## Find data and create RasterCollection from Microsoft Planetary Computer

### Import relevant modules

In [ ]:
%matplotlib inline
import arcpy
import numpy as np
from matplotlib import pyplot as plt
from arcpy import AIO
from datetime import datetime

### Find data and create RasterCollection from Microsoft Planetary Computer

In [ ]:
# Create an AIO object from the cloud storage connection file for data access
a = AIO(r'C:\AMPC_Resources\ACS_Files\esrims_pc_landsat-c2-l2.acs')

In [ ]:
# Define the query to search for images
query = {
    "collections": ["landsat-c2-l2"], # Landsat collection 2 level 2 product (https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2)
    "bbox": [-114.4345455, 36.1617946, -114.43, 36.162], # Define bounding box over Lake Mead
    "query": {"eo:cloud_cover": {"lt": 10}}, # Define cloud cover requirements
    "datetime": "1984-01-01/2023-12-31", # Define time range
    "limit": 1000, # Define max number of results to be returned
          }

In [ ]:
# Create a RasterCollection object that contains the search results
# You may see some errors due to the source images being corrupted. These images will be skipped.
rc = arcpy.ia.RasterCollection.fromSTACAPI(stac_api="https://planetarycomputer.microsoft.com/api/stac/v1",
                                  query=query,
                                  attribute_dict={
                                      "Name":"id",
                                      "Cloud Cover":"eo:cloud_cover",
                                      "StdTime":"datetime",
                                      "Platform":"platform",
                                      "Spatial Reference":"proj:epsg",
                                      "Extent": "bbox",
                                    })

In [ ]:
# Display the RasterCollection
rc

In [ ]:
# Get number of scenes from each sensor
print(f"{'Total number of scenes:' :<25} {len(rc)}")
print(f"{'Landsat 9:' :<25} {rc.getFieldValues('Platform').count('landsat-9')}")
print(f"{'Landsat 8:' :<25} {rc.getFieldValues('Platform').count('landsat-8')}")
print(f"{'Landsat 7:' :<25} {rc.getFieldValues('Platform').count('landsat-7')}")
print(f"{'Landsat 5:' :<25} {rc.getFieldValues('Platform').count('landsat-5')}")
print(f"{'Landsat 4:' :<25} {rc.getFieldValues('Platform').count('landsat-4')}")

## Data processing

### Define water detection algorithm

In [ ]:
# Detect water in Landsat using NDWI thresholding
def detect_water(item):
    
    # Define band designations for each sesnor
    if item['Platform'] == 'landsat-8':
        nir = 5
        green = 3
    else:
        nir = 4
        green = 2

    raster = item['Raster']

    # Calculate NDWI for each raster and apply a threshold to identify water pixels
    ndwi = arcpy.ia.NDWI(raster, nir_band_id = nir, green_band_id = green)
    ndwi_thres = arcpy.ia.GreaterThan(ndwi, ndwiThresh)
    out_water_mask = ndwi_thres

    # Return the thresholded NDWI image to a new RasterCollection
    return {"raster": out_water_mask, 'Name': item['Name'], "StdTime": item['StdTime']}


### Define function to identify water bodies

In [ ]:
# Identify groups of water pixels as water bodies
def water_bodies(in_ras):
    
    background_removed = arcpy.ia.SetNull(in_ras, in_ras, 'value = 0')
    
    # Using 8-neighbors pixel connectivity 
    connected_pixel = arcpy.sa.RegionGroup(background_removed, 'Eight', 'Cross', '', 0)
    count = arcpy.ia.Lookup(connected_pixel, 'Count')
    
    return count

### Perform lake extraction by year

In [ ]:
# Run the last 10 years to reduce running time
year = 2013;                #enter end year of interest
year_end = 2013;            #enter end year of interest
ndwiThresh = 0;          #change as appropriate for AOI
ndwiThresh_fraction = 0.05;  #fraction of images that meet threshold defined above (e.g. 0.3 equals 30% occurrence of greater than NDWI threshold)

# Save lake area and year information in a dictionary
area_dict = {}

while year <= year_end:
    # filter raster collection by time
    landsat_by_year = rc.filterByTime(str(year)+'-01-01', str(year)+'-12-31', date_time_format = '%Y-%m-%d')
    # detect water occurrence
    landsat_map = landsat_by_year.map(detect_water)
    # get frequency of band indices that are over threshold
    perc = landsat_map.mean()
    # get water based on frequency thresholds
    water_pixels = arcpy.ia.GreaterThan(perc, ndwiThresh_fraction)
    # get water body pixel count
    lakes = water_bodies(water_pixels)
    # save largest water body area
    area_dict[year] = lakes.maximum*30*30*1e-6
    
    #optionally save the lake raster 
    #lakes.save(r'C:\Temp\lake_mead_' + str(year) + '.tif')
    
    year += 1

## Lake Mead area change visualization

In [ ]:
# Sort by year
lists = sorted(area_dict.items())

# Extract year and lake area
x, y = zip(*lists)
x = [datetime.strptime(str(y), '%Y') for y in x]

# Plot the result
plt.plot(x, y)
plt.xlabel('Year')
plt.ylabel('Lake area ($km^2$)')
plt.show()